Update Farms rewards per block for the MEX year 5 inflation
Reduce from 200,000 MEX/block to 160,000 MEX/block

In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.parent.absolute()))
import config
from context import Context
from contracts.farm_contract import FarmContract
from utils.utils_generic import log_warning, log_step_pass, log_step_fail

context = Context()

In [ ]:
from decimal import Decimal, getcontext
import json
from tools.runners.farm_runner import get_farm_addresses_from_chain
from contracts.fees_collector_contract import FeesCollectorContract


getcontext().prec = 30

rewards_config_file = "emission_rewards_25_12_12.json"
log_warning(f"Reading rewards config from {rewards_config_file}")

# load the rewards json file
with open(rewards_config_file) as f:
    emission_rewards = json.load(f)

# set the rewards
farm_addresses = get_farm_addresses_from_chain("v2")
log_step_pass(f"Retrieved {len(farm_addresses)} farm addresses.")

total_rewards = 0

for farm_address in farm_addresses:
    farm_contract: FarmContract
    farm_contract = FarmContract.load_contract_by_address(farm_address)
    log_step_pass(f"Setting rewards for farm {farm_contract.farmToken} {farm_contract.address}")
    
    if farm_contract.farmToken not in emission_rewards:
        log_step_fail(f"Farm token {farm_contract.farmToken} not found in rewards config. Skipping.")
        continue

    new_rewards = int(Decimal(emission_rewards[farm_contract.farmToken]) * (10**18))
    log_warning(f"New rewards: {new_rewards:,}")
    total_rewards += new_rewards

    tx_hash = farm_contract.set_rewards_per_block(context.deployer_account, context.network_provider.proxy, new_rewards)
    if not context.network_provider.check_simple_tx_status(tx_hash, f"set rewards per block for: {farm_address}"):
            if not get_continue_confirmation():
                break

fees_collector_contract: FeesCollectorContract
fees_collector_contract = context.get_contracts(config.FEES_COLLECTORS)[0]

new_rewards = int(Decimal(emission_rewards["FEESCOLLECTOR"]) * (10**18))
new_rewards_per_epoch = new_rewards * (10 * 60 * 24)
total_rewards += new_rewards

log_warning(f"New fees collector rewards per epoch: {new_rewards_per_epoch:,}")
tx_hash = fees_collector_contract.set_locked_tokens_per_epoch(context.deployer_account, context.network_provider.proxy, new_rewards_per_epoch)
context.network_provider.check_simple_tx_status(tx_hash, f"set fees collector rewards per block")

print(f"Total new rewards per block: {total_rewards:,}")